# Lab 2 - First steps with Spark

At this end of this lab :

- export your notbook : File > Export Notebook As ... > Export Notebook To HTML
- An turn your cluster off !
    
## Creating a Spark session

Write in a new cell :

- `spark`  to chech if everithing is ok
- `spark._jsc.hadoopConfiguration().set("fs.s3.useRequesterPaysHeader","true")` : to access to the data

In [1]:
#Spark session
spark

# Configuraion
spark._jsc.hadoopConfiguration().set("fs.s3.useRequesterPaysHeader","true")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1618812226274_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 💾First steps with Spark - Data importation

Spark's main object class is the DataFrame, which is a distributed table. It is analogous to R's or Python (Pandas)'s data frames: one row represents an observation, one column represents a variable. But contrary to R or Python, Spark's DataFrames can be distributed over hundred of nodes.

Spark support multiple data formats, and multiple  ways to load them.

- data format : csv, json, parquet (an open source column oriented format)
- can read archive files
- schema detection or user defined schema. For static data, like a json file, schema detection can be use with good results.

Spark has multiple syntaxes to import data. Some are simple with no customisation, others are more complexes but you can specify options.

The simplest syntaxes to load a json or a csv file are :

```python
# JSON
json_df = spark.read.json([location of the file])
# csv
csv_df = spark.read.csv([location of the file])

```

In the future, you may consult the [Data Source documentation](https://spark.apache.org/docs/latest/sql-data-sources.html) to have the complete description of Spark's reading abilities.

The data you will use in this lab are real data from the twitter [sampled stream API](https://developer.twitter.com/en/docs/twitter-api/tweets/sampled-stream/introduction) and [filtered stream API](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/introduction). The tweets folder contains more than 50 files and more than 2 million tweets. The tweets was collected between the 14/04/2021 and the 18/04/2021. The total collection time was less than 10 hours.

---

### ✍Hands-on 1  - Data importation

- Load the json file store here : `s3n://spark-lab-input-data-ensai20202021/tweets/tweets20210414-143312.jsonl.gz` and name you data frame `df_tweet`

  <small> ⚙️ This file is an a `JSONL` (JSON-line) format, which means that each line of it is a JSON object. A JSON object is just a Python dictionary or a JavaScript object and looks like this: `{ key1: value1, key2: ["array", "of", "many values]}`). This file has been compressed into a `GZ` archive, hence the `.jsonl.gz` ending. Also this file is not magically appearing in your S3 storage. It is hosted on one of your teacher's bucket and has been made public, so that you can access it.</small>

- It's possible to load multiple file in a unique DataFrame. It's useful when you have daily files and want to process them all. It's the same syntax as the previous one, just specify a folder. Like `s3n://spark-lab-input-data-ensai20202021/tweets/`. Name you DataFrame `df_tweet_big`

In [3]:
# DataFrame creation
df_tweet = spark.read.json("s3n://spark-lab-input-data-ensai20202021/tweets/tweets20210414-143312.jsonl.gz")
df_tweet_big = spark.read.json("s3n://spark-lab-input-data-ensai20202021/tweets/")

# caching
df_tweet.cache()
df_tweet_big.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[auteur: string, contenu: string, date_creation: string, entities: struct<annotations:array<struct<end:bigint,normalized_text:string,probability:double,start:bigint,type:string>>,cashtags:array<struct<end:bigint,start:bigint,tag:string>>,hashtags:array<struct<end:bigint,start:bigint,tag:string>>,mentions:array<struct<end:bigint,start:bigint,username:string>>,urls:array<struct<description:string,display_url:string,end:bigint,expanded_url:string,images:array<struct<height:bigint,url:string,width:bigint>>,start:bigint,status:bigint,title:string,unwound_url:string,url:string>>>, hashtags: array<string>, like_count: bigint, other: struct<auteur_name:string>, reply_count: bigint, retweet_count: bigint]

Printing schema and some rows

In [4]:
df_tweet.show(5)
df_tweet.printSchema()
df_tweet_big.show(5)
df_tweet_big.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----------------------------+--------------------+--------------------+--------+----------+--------------------+-----------+-------------+
|        auteur|                     contenu|       date_creation|            entities|hashtags|like_count|               other|reply_count|retweet_count|
+--------------+----------------------------+--------------------+--------------------+--------+----------+--------------------+-----------+-------------+
|    Hidamaroon|かわいいー！！！ https://...|2021-04-14T12:33:...|[,,,, [[, pic.twi...|      []|         0|        [これふじう]|          0|            0|
|      azebi888|        @Asavv__ Oui va s...|2021-04-14T12:33:...|[,,, [[8, 0, Asav...|      []|         0|                [📡]|          0|            0|
|BauchaudVirgil|        Bonjour @lisalapo...|2021-04-14T12:33:...|[,,, [[20, 8, lis...|      []|         0|          [Virgil🎙]|          0|            0|
|   XIAOC4TB0Y_|        @kiribraincell ya...|2021-04-14T12:33:...|[,,, [[14, 0, kir..

## 🥉Data frame basic manipulations

If DataFrames are immutable, they can however be **_transformed_** in other DataFrames, in the sense that a modified copy is returned. Such **transformations** include: filtering, sampling, dropping columns, selecting columns, adding new columns...

First, you can get information about the columns with:

```python
df.columns       # get the column names
df.schema        # get the column names and their respective type
df.printSchema() # same, but human-readable
```

You can select columns with the `select()` method. It takes as argument a list of column name. For example :

```python
df_with_less_columns = df\
  .select("variable3","variable_four","variable-6")

# Yes, you do need the ugly \ at the end of the line,
# if you want to chain methods between lines in Python
```

You can get nested columns easily with :

```python
df.select("parentField.nestedField")
```

To filter data you could use the `filter()` method. It take as input an expression that gets evaluated for each observation and should return a boolean. Sampling is performed with the `sample()` method. For example :

```python
df_with_less_rows = df\
  .sample(fraction=0.001)\
  .filter(df.variable1=="value")\
  .show(10)
```

As said before your data are distributed over multiple nodes (executors) and data inside a node are split into partitions. Then each transformations will be run in parallel. They are called *narrow transformation* For example, to sample a DataFrame, Spark sample every partitions in parallel because sample all partition produce the sample DataFrame. For some transformations, like `groupBy()` it's impossible, and it's cannot be run in parallel.

![](https://raw.githubusercontent.com/HealerMikado/panorama_big_data_2021/main/labs/lab%202%20-%20first%20steps%20with%20Spark/img/spark_exemple1_pipeline.png)

<!-- take() collect() limit() first() show() -->
<!-- lien vers la doc https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#dataframe-apis -->

### 😴Lazy evaluation

This is because Spark has what is known as **lazy evaluation**, in the sense that it will wait as much as it can before performing the actual computation. Said otherwise, when you run an instruction such as:

```python
tweet_author_hashtags = df_tweet_big.select("auteur","hashtags")
```

... you are not executing anything! Rather, you are building an **execution plan**, to be realised later.

Spark is quite extreme in its laziness, since only a handful of methods called **actions**, by opposition to **transformations**, will trigger an execution. The most notable are:

1. `collect()`, explicitly asking Spark to fetch the resulting rows instead of to lazily wait for more instructions,
2. `take(n)`, asking for `n` first rows
3. `first()`, an alias for `take(1)`
4. `show()` and `show(n)`, human-friendly alternatives[^5]
5. `count()`, asking for the numbers of rows
6. all the "write" methods (write on file, write to database), see [here](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#input-and-output) for the list

[^5]: `first()` is exactly `take(1)` ([ref]( https://stackoverflow.com/questions/37495039/difference-between-spark-rdds-take1-and-first)) and show prints the result instead of returning it as a list of rows ([ref](https://stackoverflow.com/questions/53884994/what-is-the-difference-between-dataframe-show-and-dataframe-take-in-spark-t))

**This has advantages:** on huge data, you don't want to accidently perform a computation that is not needed. Also, Spark can optimize each **stage** of the execution in regard to what comes next. For instance, filters will be executed as early as possible, since it diminishes the number of rows on which to perform later operations. On the contrary, joins are very computation-intense and will be executed as late as possible. The resulting **execution plan** consists in a **directed acyclic graph** (DAG) that contains the tree of all required actions for a specific computation, ordered in the most effective fashion.

**This has also drawbacks.** Since the computation is optimized for the end result, the intermediate stages are discarded by default. So if you need a DataFrame multiple times, you have to cache it in memory because if you don't Spark will recompute it every single time. 

---

### ✍Hands-on 2 - Data frame basic manipulations

- How many rows have your two DataFrame ?

In [7]:
print(f'the small DF has {df_tweet.count()} rows')
print(f'the big DF has {df_tweet_big.count()} rows')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

the small DF has 1000 rows
the big DF has 2588000 rows

- Sample `df_tweet_big` and keep only 10% of it. Create a new DataFrame named `df_tweet_sampled`. If computations take too long on the full DataFrame, use this one instead or add a sample transformation in your expression. 

In [9]:
df_tweet_sampled = df_tweet_big.sample(fraction=0.1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- Define a DataFrame `tweet_author_hashtags`  with only the `auteur` and `hashtags` columns

In [10]:
tweet_author_hashtags = df_tweet_big.select("auteur", "hashtags")
tweet_author_hashtags.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------+
|         auteur|hashtags|
+---------------+--------+
|   Aporante1150|      []|
|         sr8884|      []|
|      runto_613|      []|
|         R1___y|      []|
|JlGCfVHMhl4tn0Q|      []|
+---------------+--------+
only showing top 5 rows

- Print (few lines of) a DataFrame with only the `auteur`, `mentions`, and `urls` columns. (`mentions` and `urls` are both nested columns in `entities`.)

In [11]:
df_tweet_big.select("auteur", "entities.urls", "entities.mentions").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----+--------+
|         auteur|urls|mentions|
+---------------+----+--------+
|   Aporante1150|null|    null|
|         sr8884|null|    null|
|      runto_613|null|    null|
|         R1___y|null|    null|
|JlGCfVHMhl4tn0Q|null|    null|
+---------------+----+--------+
only showing top 5 rows

- Filter your first DataFrame and keep only tweets with more than 1 like. Give a name for this new, transformed DataFrame and print. Print (few lines of) it.

In [13]:
df_tweet_big.filter(df_tweet_big.like_count > 0).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------------------------+--------------------+-----------------------+----------------------------------+----------+-----------------------------+-----------+-------------+
|       auteur|                             contenu|       date_creation|               entities|                          hashtags|like_count|                        other|reply_count|retweet_count|
+-------------+------------------------------------+--------------------+-----------------------+----------------------------------+----------+-----------------------------+-----------+-------------+
|       R1___y|どっかのサブマネやりたいけど
メイ...|2021-04-18T08:20:...|                   null|                                []|         1|[固ツイにリプして！𝚈𝚄𝚁?...|          0|            0|
|   Ryoutaxx19|休みが終わっちゃうなぁ
ムラムラを...|2021-04-18T08:20:...|[,, [[48, 35, 裏アカ...|[裏アカ女子とつながりたい, 裏ア...|         1|                     [Ryo-🌵]|          0|            0|
| Cigar_smoke_| 동생넘어졌대소 얼레벌레 학교체육...|2021-04-18T08:20:...|    

## 🥈Basic DataFrame column manipulation 

You can add/update/rename column of a DataFrame with spark :

- Drop : `df.drop(columnName : str )`
- Rename : `df.withColumnRenamed(oldName : str, newName : str)`
- Add/update : `df.withColumn(columnName : str, columnExpression)` 

For example

```python
tweet_df_with_like_rt_ratio = tweet_df\
  .withColumn(        # computes new variable
    "like_rt_ratio", # like_rt_ratio "OVERCONFIDENCE"
    (tweet_df.like_count /flights.retweet_count
   )

```

See [here](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#functions) for the list of all functions available in an expression.

### ✍Hands-on 3 - Basic DataFrame column manipulation  

- Define a DataFrame with a column names `interaction_count`. This column is the sum of `like_count`, `reply_count` and `retweet_count`.

In [15]:
df_tweet_big_interation = df_tweet_big.withColumn("interaction_count", df_tweet_big.like_count+df_tweet_big.reply_count+df_tweet_big.retweet_count )
df_tweet_big_interation.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------------------------------+--------------------+--------+--------+----------+-----------------------------+-----------+-------------+-----------------+
|         auteur|                              contenu|       date_creation|entities|hashtags|like_count|                        other|reply_count|retweet_count|interaction_count|
+---------------+-------------------------------------+--------------------+--------+--------+----------+-----------------------------+-----------+-------------+-----------------+
|   Aporante1150|       そういや皐月賞だったのか今日は|2021-04-18T08:20:...|    null|      []|         0|                 [アポロ🌵🐕]|          0|            0|                0|
|         sr8884|                 صباح أول اختبار ب...|2021-04-18T08:20:...|    null|      []|         0|         [شجيِّن‌‌عبدالعزي...|          0|            0|                0|
|      runto_613|ふぅ〜！！！！！！！！！！！！！！...|2021-04-18T08:20:...|    null|      []|         0|                   [こんよう]|      

- Update the DataFrame you imported at the beginning of this lab and drop the `other` column

In [16]:
df_tweet_big = df_tweet_big.drop("other")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 🥇Advance DataFrame column manipulation 

### 🥽Array manipulation

Some columns often contain arrays (lists) of values instead of just one value. This may seem surprising but this actually quite natural. For instance, you may create an array of words from a text, or generate a list of random numbers for each observation, etc.

You may **create array of values** with:

- `split(text : string, delimiter : string)`, turning a text into an array of strings

You may **use array of values** with:

- `size(array : Array)`, getting the number of elements

- `array_contains(inputArray : Array, value : any)`, checking if some value appears

- `explode(array : Array)`, unnesting an array and duplicating other values. For instance it if use `explode()` over the hashtags value of this DataFrame:

  | Auteur | Contenu                             | Hashtags         |
  | ------ | ----------------------------------- | ---------------- |
  | Bob    | I love #Spark and #bigdata          | [Spark, bigdata] |
  | Alice  | Just finished #MHrise, best MH ever | [MHrise]         |

  I will get :

  | Auteur | Contenu                             | Hashtags         | Hashtag |
  | ------ | ----------------------------------- | ---------------- | ------- |
  | Bob    | I love #Spark and #bigdata          | [Spark, bigdata] | Spark   |
  | Bob    | I love #Spark and #bigdata          | [Spark, bigdata] | bigdata |
  | Alice  | Just finished #MHrise, best MH ever | [MHrise]         | MHrise  |

  

All this function must be imported first :

```python
from pyspark.sql.functions import split, explode, size, array_contains
```

Do not forget, to create a new column, you should use `withColumn()`. For example : 

```python
df.withColumn("new column", explode("array"))
```

#### ✍Hands-on 4 - Array manipulation 

- Keep all the tweets with hashtags and for each remaining line, split the hashtag text into an array of hashtags

In [18]:
from pyspark.sql.functions import split, explode, size, array_contains

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_tweet_big.filter(size("hashtags") > 0).withColumn("hashtag", explode("hashtags")).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------------------------------+--------------------+-------------------------+---------------------------------+----------+-----------+-------------+------------------+
|     auteur|                            contenu|       date_creation|                 entities|                         hashtags|like_count|reply_count|retweet_count|           hashtag|
+-----------+-----------------------------------+--------------------+-------------------------+---------------------------------+----------+-----------+-------------+------------------+
|    1262422|みんなからの匿名質問を募集中！

...|2021-04-18T08:20:...|[[[21, さちこさん, 0.6...|         [質問箱, 匿名質問募集中]|         0|          0|            0|            質問箱|
|    1262422|みんなからの匿名質問を募集中！

...|2021-04-18T08:20:...|[[[21, さちこさん, 0.6...|         [質問箱, 匿名質問募集中]|         0|          0|            0|    匿名質問募集中|
|ifamart0392|               RT @BabyBujingjin...|2021-04-18T08:20:...|     [,, [[135, 130, S...|                           [SB19]|     

- Create a new column with the number of words of the `contenu` column. (Use `split()` + `size()`)

In [23]:
df_tweet_big.withColumn("word_count", size(split("contenu", " "))).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------------------------------+--------------------+--------+--------+----------+-----------+-------------+----------+
|         auteur|                              contenu|       date_creation|entities|hashtags|like_count|reply_count|retweet_count|word_count|
+---------------+-------------------------------------+--------------------+--------+--------+----------+-----------+-------------+----------+
|   Aporante1150|       そういや皐月賞だったのか今日は|2021-04-18T08:20:...|    null|      []|         0|          0|            0|         1|
|         sr8884|                 صباح أول اختبار ب...|2021-04-18T08:20:...|    null|      []|         0|          0|            0|         9|
|      runto_613|ふぅ〜！！！！！！！！！！！！！！...|2021-04-18T08:20:...|    null|      []|         0|          0|            0|         1|
|         R1___y| どっかのサブマネやりたいけど
メイ...|2021-04-18T08:20:...|    null|      []|         1|          0|            0|         1|
|JlGCfVHMhl4tn0Q|                 Иногда люди н

- Count how many tweet contain the `covid19` hashtag (use the `count()` action)

In [26]:
df_tweet_big.filter(array_contains("hashtags", "COVID19")).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9238

### 🥼User defined function

For more very specific column manipulation you will need Spark's `udf()` function (*User Defined Function*). It can be useful if you Spark does not provide a feature you want. But Spark is a popular and active project, so before coding an udf, go check the documentation. For instance for natural language processing, Spark already has some [functions](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.feature.Tokenizer.html#pyspark.ml.feature.Tokenizer). Last things, python udf can lead to performance issues (see https://stackoverflow.com/a/38297050) and learning a little bit of scala or java can be a good idea.

For example :

```python
# !!!! DOES NOT WORK !!!!
def to_lower_case(string):
	return string.lower()
	
df.withColumn("tweet_lower_case", to_lower_case(df.contenu))
```

will just crash. Keep in mind that Spark is a distributed system, and that Python is only installed on the central node, as a convenience to let you execute instructions on the executor nodes. But by default, pure Python functions can only be executed where Python is installed! We need `udf()` to enable Spark to send Python instructions to the worker nodes.

Let us see how it is done :

```python
# imports
from pyspark.sql.functions import udf
from pyspark.sql.functions import explode
from pyspark.sql.types import StringType

# pure python functions
def to_lower_case(string):
    return string.lower()

# user definid function
to_lower_case_udf = udf(
    lambda x: to_lower_case(x), StringType()
) #we use a lambda function to create the udf.

# df manipulation
df_tweet_small\
  .select("auteur","hashtags")\
  .filter("size(hashtags)!=0")\
  .withColumn("hashtag", explode("hashtags"))\
  .withColumn("hashtag", to_lower_case_udf("hashtag")).show(10)
```

---

#### ✍Hands-on 5 - User defined function 

- Create an user defined function that counts how many words a tweet contains. (your function will return an `IntegerType` and not a `StringType`)

In [27]:
# imports
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# pure python functions
def word_count(string):
    return len(string.split(" "))

# user definid function
word_count_udf = udf(
    lambda x: word_count(x), IntegerType()
) #we use a lambda function to create the udf.

# df manipulation
df_tweet_big\
  .withColumn("word_count", word_count_udf("contenu")).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------------------------------+--------------------+--------------------+--------+----------+-----------+-------------+----------+
|         auteur|                              contenu|       date_creation|            entities|hashtags|like_count|reply_count|retweet_count|word_count|
+---------------+-------------------------------------+--------------------+--------------------+--------+----------+-----------+-------------+----------+
|   Aporante1150|       そういや皐月賞だったのか今日は|2021-04-18T08:20:...|                null|      []|         0|          0|            0|         1|
|         sr8884|                 صباح أول اختبار ب...|2021-04-18T08:20:...|                null|      []|         0|          0|            0|         9|
|      runto_613|ふぅ〜！！！！！！！！！！！！！！...|2021-04-18T08:20:...|                null|      []|         0|          0|            0|         1|
|         R1___y| どっかのサブマネやりたいけど
メイ...|2021-04-18T08:20:...|                null|      []|         1| 

## 🔩Aggregation functions

Spark offer a variety of aggregation functions :

- `count(column : string)` will count every not null value of the specify column. You cant use `count(1)` of `count("*")` to count every line (even row with only null values)

- `countDisctinct(column : string)` and `approx_count_distinct(column : string, percent_error: float)`. If the exact number is irrelevant, `approx_count_distinct()`should be preferred.

  Counting distinct elements cannot be done in parallel, and need a lot data transfer. But if you only need an approximation, there is a algorithm, named hyper-log-log (more info [here](https://databricks.com/fr/blog/2016/05/19/approximate-algorithms-in-apache-spark-hyperloglog-and-quantiles.html)) that can be parallelized. 

  ```python
  from pyspark.sql.functions import count, countDistinct, approx_count_distinct
  
  df.select(count("col1")).show()
  df.select(countDistinct("col1")).show()
  df.select(approx_count_distinct("col1"), 0.1).show()
  ```

- You have access to all other common functions `min()`, `max()`, `first()`, `last()`, `sum()`, `sumDistinct()`, `avg()` etc (you should import them first `from pyspark.sql.functions import min, max, avg, first, last, sum, sumDistinct`) 

---

### ✍Hands-on 6 - Aggregation functions

- What are the min, max, average of `interaction_count`

In [29]:
from pyspark.sql.functions import min, max, avg, first, last, sum, sumDistinct

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_tweet_big_interation.select(min("interaction_count"),max("interaction_count"),avg("interaction_count")).first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(min(interaction_count)=0, max(interaction_count)=2670900, avg(interaction_count)=2111.469003863988)

- How many tweets have hashtags ? Distinct hashtags ? Try the approximative count with 0.1 and 0.01as maximum estimation error allowed.

In [39]:
from pyspark.sql.functions import count, countDistinct, approx_count_distinct

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df_tweet_big.select(count("hashtags"), countDistinct("hashtags"), approx_count_distinct("hashtags", 0.1), approx_count_distinct("hashtags",0.01)).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------------+-------------------------------+-------------------------------+
|count(hashtags)|count(DISTINCT hashtags)|approx_count_distinct(hashtags)|approx_count_distinct(hashtags)|
+---------------+------------------------+-------------------------------+-------------------------------+
|        2588000|                  141194|                         144632|                         139760|
+---------------+------------------------+-------------------------------+-------------------------------+

## Grouping functions

Like SQL you can group row by a criteria with Spark. Just use the `groupBy(column : string)` method. Then you can compute some aggregation over those groups.

```python
df.groupBy("col1").agg(
  count("col2").alias("quantity") # alias is use to specify the name of the new column
).show() 
```

The `agg()` method can take multiples argument to compute multiple aggregation at once.

```python
df.groupBy("col1").agg(
	count("col2").alias("quantity"), min("col2").alias("min"), avg("col3").alias("avg3") ).show()
```

Aggregation and grouping transformations work differently than the previous method like `filter()`, `select()`, `withColumn()` etc. Those transformations cannot be run over each partitions in parallel, and need to transfer data between partitions and executors.  They are called "wide transformations"

<img src="https://raw.githubusercontent.com/HealerMikado/panorama_big_data_2021/main/labs/lab%202%20-%20first%20steps%20with%20Spark/img/spark_exemple2_pipeline.png" style="zoom:30%;" />
---

### ✍Hands-on 7 - Grouping functions

- Compute a daframe with the min, max and average retweet of each `auteur`. Then order it by the max number of retweet in descending order by . To do that you can use the following syntax

  ```python
  from pyspark.sql.functions import desc
  df.orderBy(desc("col"))
  ```

In [48]:
from pyspark.sql.functions import desc
df_tweet_big.groupBy("auteur").agg(min("retweet_count").alias("min_RT"), max("retweet_count").alias("max_RT"), avg("retweet_count").alias("avg_RT")).orderBy(desc("max_RT")).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------+-------+---------+
|     auteur| min_RT| max_RT|   avg_RT|
+-----------+-------+-------+---------+
|liamsbestie|2670900|2670900|2670900.0|
|savage_hazz|2670257|2670257|2670257.0|
|     lrtsee|2670241|2670241|2670241.0|
|   Bjk_km13|1466240|1466240|1466240.0|
|  Kimkati15|1465141|1465141|1465141.0|
+-----------+-------+-------+---------+
only showing top 5 rows

## 🔌Spark SQL

Spark understand SQL statement. It's not a hack nor a workaround to use SQL in Spark, it's one a the more powerful feature in Spark. To use SQL in you need :

1. Register a view pointing to your DataFrame

   ```python
   my_df.createOrReplaceTempView(viewName : str)
   ```

2. Use the sql function

   ```python
   spark.sql("""
   You sql statment
   """)
   ```

   You could manipulate every registered DataFrame by their view name with plain SQL.

In fact you can do most of this tutorial without any knowledge in PySpark nor Spark. Lot of things can be done in Sparkk only by only knowing SQL and how to use it in Spark. 

### ✍Hands-on 8 - Spark SQL

- How many tweets have hashtags ? Distinct hashtags ? 

In [61]:
df_tweet_big.select("contenu", "hashtags").createOrReplaceTempView("view_hashtag_content")

spark.sql("""
SELECT COUNT(*), COUNT(DISTINCT(contenu))
FROM view_hashtag_content
WHERE size(hashtags) > 0
""").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------------+
|count(1)|count(DISTINCT contenu)|
+--------+-----------------------+
|  459675|                 343802|
+--------+-----------------------+

- Compute a dataframe with the min, max and average retweet of each `auteur` using Spark SQL

In [66]:
df_tweet_big.createOrReplaceTempView("view_tweet_big")
spark.sql("""
SELECT min(retweet_count), max(retweet_count), avg(retweet_count)
FROM view_tweet_big
GROUP BY auteur
ORDER BY MAX(retweet_count) DESC
""").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------------------+
|min(retweet_count)|max(retweet_count)|avg(retweet_count)|
+------------------+------------------+------------------+
|           2670900|           2670900|         2670900.0|
|           2670257|           2670257|         2670257.0|
|           2670241|           2670241|         2670241.0|
|           1466240|           1466240|         1466240.0|
|           1465141|           1465141|         1465141.0|
+------------------+------------------+------------------+
only showing top 5 rows

## Joins in Spark

Like a SQL, Spark can join two dataset  by comparing the value of one or more keys using joins. Joins are by nature wide transformation, so data will be transferred between executors and will take time. But Spark will distinct two cases :

- Big table to big table join
- Big table to small table join
- (The case small table to small table is irrelevant for big data)

And optimize the join according to the actual case. (For more information Spark: The Definitive Guide pages 148 - 151)

Doing a join is pretty easy. You need :

- At least two DataFrames (obviously) with columns with the same keys
- a join expression
- the join transformation

For instance :

```python
# Creation of 3 small DF
person=spark.createDataFrame([
    (0,"Bill Chambers",0,[100])
    ,(1,"Matei Zaharia",1,[500,250,100])
    ,(2,"Michael Armbrust",1,[250,100])])\
	.toDF("id","name","graduate_program","spark_status")
graduateProgram=spark.createDataFrame([
    (0,"Masters","School of Information","UC Berkeley")
    ,(2,"Masters","EECS","UC Berkeley"),(1,"Ph.D.","EECS","UC Berkeley")])\
	.toDF("id","degree","department","school")
sparkStatus=spark.createDataFrame([
    (500,"Vice President")
    ,(250,"PMC Member")
    ,(100,"Contributor")])\
	.toDF("id","status")
    
# A join expression
joinExpression=person["graduate_program"]==graduateProgram['id']

# The join transformation in action
person.join(graduateProgram, joinExpression).show()
```

By default Spark compute inner joins, but you can pass a third argument to the join transformation with its type. You can do :

- Inner joins : by default or with the "inner" argument
- Outer joins : "outer" argument
- Left / right outer joins : "left_outer", "right_outer" argument
- Left semi join : it's more a filter than a join. It only keep the row in the left DataFrame that have a match in the right DataFrame. Use he "left_semi" argument
- Left anti join : The opposite of the previous one. Use he "left_anti" argument
- You can do cross joins to, but it's a very bad idea to do so, so please don't !

### ✍ Hands-on 9 - Joins in Spark

- Import the files stored in `s3n://spark-lab-input-data-ensai20202021/users/` in a DataFrame  and its informations to your DataFrame. Filter your new DataFrame to only keep verified user (`verified == True`) and group by user and get the most active user of your DataFrame.

In [93]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, TimestampType, BooleanType
schema = StructType([ \
    StructField("created_at",TimestampType(),True), \
    StructField("id",StringType(),True), \
    StructField("name",StringType(),True), \
    StructField("username", StringType(), True), \
    StructField("withheld",StructType([ 
        StructField("country_codes", ArrayType(StringType(),True)),\
        StructField("scope", StringType(), True)  \
    ])),
    StructField("verified",BooleanType(),True)             
  ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [101]:
df_user_big = spark.read.json("s3n://spark-lab-input-data-ensai20202021/users/")
df_user_big.cache()
df_user_big.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+--------------------+---------------+--------+--------+
|          created_at|                 id|                name|       username|verified|withheld|
+--------------------+-------------------+--------------------+---------------+--------+--------+
|2019-10-14T07:29:...|1183645830277992448|Diamond Plastic C...|plastic_company|   false|    null|
|2020-07-03T12:57:...|1279036490429771776|            安田流佳|Q1oApaNoqiekjcQ|   false|    null|
|2021-01-25T22:56:...|1353839203029643265|       ❄침대(파도)🌊|  twinkle_wave_|   false|    null|
|2016-05-21T10:35:...| 733969482243178497|   エリンギ語録Botン| eringi_kusobot|   false|    null|
|2020-06-17T13:27:...|1273245870130032642|ยิ้นนี้โลกละลาย🍼...|   Israporn9675|   false|    null|
+--------------------+-------------------+--------------------+---------------+--------+--------+
only showing top 5 rows

In [102]:
df_user_big.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- withheld: struct (nullable = true)
 |    |-- country_codes: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- scope: string (nullable = true)

In [97]:
from pyspark.sql.functions import to_date, lit
df_user_big.filter("created_at" > to_date(lit("2019-01-01"))).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2565000

In [98]:
df_user_big.filter("verified").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

567

In [99]:
joinExpression=df_user_big["username"]==df_tweet_big['auteur']

df_user_big.join(df_tweet_big, joinExpression).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+----------------------+-------------+--------+--------+-------------+-------------------------------------+--------------------+------------------------------+---------------------------------+----------+-----------+-------------+
|         created_at|                 id|                  name|     username|withheld|verified|       auteur|                              contenu|       date_creation|                      entities|                         hashtags|like_count|reply_count|retweet_count|
+-------------------+-------------------+----------------------+-------------+--------+--------+-------------+-------------------------------------+--------------------+------------------------------+---------------------------------+----------+-----------+-------------+
|2018-03-17 12:13:38| 974982086368129024|              Aqua🦋🟥|    00020002L|    null|    null|    00020002L|                 RT @FantaCarrot_2...|2021-04-18T14:43:...|          [,,, [[

In [100]:
df_user_big.join(df_tweet_big, joinExpression).filter("verified").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12385

**DO NOT FORGET TO TURN YOUR CLUSTER OFF!**

**DO NOT FORGET TO TURN YOUR CLUSTER OFF!**

**DO NOT FORGET TO TURN YOUR CLUSTER OFF!**

**DO NOT FORGET TO TURN YOUR CLUSTER OFF!**

**DO NOT FORGET TO TURN YOUR CLUSTER OFF!**

**DO NOT FORGET TO TURN YOUR CLUSTER OFF!**

**DO NOT FORGET TO TURN YOUR CLUSTER OFF!**

**DO NOT FORGET TO TURN YOUR CLUSTER OFF!**